<a href="https://colab.research.google.com/github/manglamkumar0621/covid19/blob/secondary/covid19_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To install modules/packages in runtime

In [0]:
#!pip install geocoder

Importing all necessary modules

In [0]:
import folium
from folium.plugins import HeatMap
from folium.features import GeoJson
import json
import pandas as pd
import matplotlib.pyplot as plt
import io
from google.colab import files
#import geocoder
%matplotlib inline

To read a file from local drive:

In [4]:
#from google.colab import files
uploaded1 = files.upload()

Saving district_coordinates.csv to district_coordinates.csv
Saving covid19_districtwise20200422174142.csv to covid19_districtwise20200422174142.csv
Saving covid19_20200422174042.csv to covid19_20200422174042.csv


Read the input files and assign local variable to them

In [0]:
#import io
district_coordinates = pd.read_csv(io.BytesIO(uploaded1['district_coordinates.csv']))
covid19_districtwise = pd.read_csv(io.BytesIO(uploaded1['covid19_districtwise20200422174142.csv']))
covid19_statetwise = pd.read_csv(io.BytesIO(uploaded1['covid19_20200422174042.csv']))

In [87]:
district_coordinates.head(), covid19_districtwise.head(), covid19_statetwise.head()

(         STATE DISTRICT             COORDINATES
 0  Maharashtra   Mumbai  19.0760° N, 72.8777° E
 1  Maharashtra     Pune  18.5204° N, 73.8567° E
 2  Maharashtra    Thane  19.2183° N, 72.9781° E
 3  Maharashtra  Palghar  19.6967° N, 72.7699° E
 4  Maharashtra   Nashik  19.9975° N, 73.7898° E,
          STATE DISTRICT CONFIRMED
 0  Maharashtra   Mumbai     3,451
 1  Maharashtra     Pune       716
 2  Maharashtra    Thane       447
 3  Maharashtra  Palghar       129
 4  Maharashtra   Nashik        95,
          State Confirmed Active Recovery Deceased
 0  Maharashtra     5,218  4,245      722      251
 1      Gujarat     2,272  2,033      144       95
 2        Delhi     2,156  1,498      611       47
 3    Rajasthan     1,868  1,513      328       27
 4   Tamil Nadu     1,596    943      635       18)

# **Data- preprocessing:**

1. Drop all records for which coordinates are not available. For some covid cases, exact place is not available.

In [88]:
district_coordinates.dropna(subset=['COORDINATES'], inplace= True)
district_coordinates.isna().sum()

STATE          0
DISTRICT       0
COORDINATES    0
dtype: int64

2. Split coordinates into longitude and latitude. 

In [90]:
district_coordinates[['LONGITUDE','LATITUDE']] = district_coordinates.COORDINATES.str.split(pat=',',expand = True)
district_coordinates

,STATE,DISTRICT,COORDINATES,LONGITUDE,LATITUDE
0,Maharashtra,Mumbai,"19.0760° N, 72.8777° E",19.0760° N,72.8777° E
1,Maharashtra,Pune,"18.5204° N, 73.8567° E",18.5204° N,73.8567° E
2,Maharashtra,Thane,"19.2183° N, 72.9781° E",19.2183° N,72.9781° E
3,Maharashtra,Palghar,"19.6967° N, 72.7699° E",19.6967° N,72.7699° E
4,Maharashtra,Nashik,"19.9975° N, 73.7898° E",19.9975° N,73.7898° E
...,...,...,...,...,...
433,Manipur,Imphal East,"24.7807° N, 93.9674° E",24.7807° N,93.9674° E
434,Tripura,North Tripura,"24.0797° N, 92.2630° E",24.0797° N,92.2630° E
435,Tripura,Gomati,"23.5167° N, 91.6372° E",23.5167° N,91.6372° E
436,Arunachal Pradesh,Lohit,"27.9040° N, 96.1739° E",27.9040° N,96.1739° E


3. Getting numerical values of Longitute and Latitude. Removing COORDINATES column.


In [0]:
district_coordinates['LONGITUDE'] = district_coordinates['LONGITUDE'].apply(lambda x : x.replace('° N',''))
district_coordinates['LATITUDE'] = district_coordinates['LATITUDE'].apply(lambda x : x.replace('° E',''))
district_coordinates= district_coordinates.drop(['COORDINATES'], axis= 1)

In [92]:
district_coordinates

,STATE,DISTRICT,LONGITUDE,LATITUDE
0,Maharashtra,Mumbai,19.0760,72.8777
1,Maharashtra,Pune,18.5204,73.8567
2,Maharashtra,Thane,19.2183,72.9781
3,Maharashtra,Palghar,19.6967,72.7699
4,Maharashtra,Nashik,19.9975,73.7898
...,...,...,...,...
433,Manipur,Imphal East,24.7807,93.9674
434,Tripura,North Tripura,24.0797,92.2630
435,Tripura,Gomati,23.5167,91.6372
436,Arunachal Pradesh,Lohit,27.9040,96.1739
